In [1]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from sklearn.preprocessing import LabelEncoder
from underthesea import word_tokenize

In [2]:
path = './balanced_data.csv'
def readdata(path):
    list_file = os.listdir(path)
    data = pd.DataFrame()
    for filename in list_file:        
        data = pd.concat([data, pd.read_csv(os.path.join(path, filename), sep=',')])
    return data['content'], data['label']

In [3]:

def remove_emoji(text):
    for emoji in UNICODE_EMOJI.values():
        text = text.replace(emoji, "")
    for emoticon in EMOTICONS_EMO.values():
        text = text.replace(emoticon, "")
    return text

def clean_text(text):
    text = text.lower()
    text = remove_emoji(text) 
    text = re.sub(r'\d+', ' ', text)  
    text = re.sub(r'[^\w\s]', ' ', text)  
    text = re.sub(r'\s+', ' ', text).strip()  
    
    words = word_tokenize(text, format="text")
    return words

def preprocess_data(file_csv):
    df = pd.read_csv(file_csv)
    
    df = df[['content', 'label']].dropna()

    df['content'] = df['content'].fillna('')
    df['content'] = df['content'].apply(clean_text)

    label_encoder = LabelEncoder()
    df['label'] = label_encoder.fit_transform(df['label'])

    return df, label_encoder

file_csv = './balanced_data.csv'
data, label_encoder = preprocess_data(file_csv)

reviews = data['content'].tolist()
labels = data['label'].tolist()
reviews

['túi ưng_ý lắm',
 'mùi thơm dễ_chịu',
 'chất_lượng sản_phẩm rất kém hàng bị lỗi tưa đường chỉ từ trên xuống dưới',
 'sp quá tệ',
 'giá tiền đi_đôi với chất_lượng nói_chung k nên mua mặc_tí đã rách tứ_tung mà bạn nào dưới kg mới vừa nhé_k có mà trật_ních mua về nhớ mang đi may lại toàn_bộ',
 'shop giải_thích cho mình đc ko',
 'hàng_không đúng mẫu shop phục_vụ tạm được',
 'mặc_tiện cho con bú',
 'thời_gian giao hàng tạm được',
 'lại đặt tiếp thêm cái nữa em nhé',
 'trên cả tuyệt_vời',
 'shop giao hàng cực nhanh',
 'm mua kiểu áo khác nhau nhưng áo nào vải cũng rất đẹp',
 'giá_cả chấp_nhận được',
 'chất_lượng sản_phẩm tạm được giá_cả chấp_nhận được thời_gian giao hàng rất nhanh',
 'hàng cực đẹp đẹp hơn trên ảnh nhiều chủ_shop tận_tâm nhất mà mình từng gặp',
 'chất_lượng sản_phẩm tuyệt_vời',
 'vải đẹp chắc_chắn hợp giá tiền',
 'm kg mặc ôm ko đc thoải_mái hơi buồn',
 'sản_phẩm tạm được',
 'bảo cường_lực mà ship cho nilon dán màn đã thế lại còn k dán được vào đt',
 'hang ko giong hinh',
 '

In [4]:
file_path = 'baomoi.model.bin'

try:
    model_embedding = KeyedVectors.load_word2vec_format(file_path, binary=True)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")

Model loaded successfully!


In [5]:
word_labels = list(model_embedding.key_to_index.keys())
max_seq = 100 
embedding_size = 400

def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = comment.split()
    for i in range(min(len(words), max_seq)):
        word = words[i]
        if word in word_labels:
            matrix[i] = model_embedding[word]
    return matrix



In [6]:
train_data = [comment_embedding(review) for review in tqdm(reviews)]
train_data = np.array(train_data, dtype=np.float32)

label_data = []
for label in tqdm(labels):
    label_vec = np.zeros(3)
    try:
        label_vec[int(label)] = 1
    except:
        label_vec[0] = 1
    label_data.append(label_vec)
label_data = np.array(label_data)

x_train, x_test, y_train, y_test = train_test_split(train_data, label_data, test_size=0.2, random_state=42)



100%|██████████| 18351/18351 [00:00<00:00, 646076.46it/s]


In [7]:
# 6. Chuẩn bị dữ liệu cho CNN
x_train = x_train.reshape(x_train.shape[0], max_seq, embedding_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], max_seq, embedding_size, 1).astype('float32')


In [8]:

# 7. Xây dựng mô hình CNN
num_classes = 3
filter_sizes = 3
num_filters = 150
dropout_rate = 0.5

model = keras.Sequential([
    layers.Conv2D(num_filters, (filter_sizes, embedding_size), activation='relu', input_shape=(max_seq, embedding_size, 1)),
    layers.MaxPooling2D(pool_size=(max_seq - filter_sizes + 1, 1)),
    layers.Dropout(dropout_rate),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:

# 8. Huấn luyện mô hình
adam = tf.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

print(model.summary())
model.fit(x=x_train, y=y_train, batch_size=30, epochs=50, validation_data=(x_train, y_train))
model.save('models.h5')



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 1, 150)     │       180,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 1, 1, 150)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 1, 150)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        19,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 199,865 (780.72 KB)

 Trainable params: 199,865 (780.72 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/50
490/490 ━━━━━━━━━━━━━━━━━━━━ 42s 82ms/step - accuracy: 0.5184 - loss: 1.5146 - val_accuracy: 0.6282 - val_loss: 0.7636
Epoch 2/50
490/490 ━━━━━━━━━━━━━━━━━━━━ 38s 78ms/step - accuracy: 0.6154 - loss: 0.8460 - val_accuracy: 0.6407 - val_loss: 0.7994
Epoch 3/50
490/490 ━━━━━━━━━━━━━━━━━━━━ 35s 72ms/step - accuracy: 0.6152 - loss: 0.8444 - val_accuracy: 0.6163 - val_loss: 0.8728
Epoch 4/50
490/490 ━━━━━━━━━━━━━━━━━━━━ 34s 70ms/step - accuracy: 0.5862 - loss: 0.9576 - val_accuracy: 0.6686 - val_loss: 0.7783
Epoch 5/50
490/490 ━━━━━━━━━━━━━━━━━━━━ 36s 73ms/step - accuracy: 0.6212 - loss: 0.8846 - val_accuracy: 0.6611 - val_loss: 0.7841
Epoch 6/50
490/490 ━━━━━━━━━━━━━━━━━━━━ 35s 71ms/step - accuracy: 0.6110 - loss: 0.9013 - val_accuracy: 0.5677 - val_loss: 0.8908
Epoch 7/50
490/490 ━━━━━━━━━━━━━━━━━━━━ 36s 73ms/step - accuracy: 0.6343 - loss: 0.8496 - val_accuracy: 0.6715 - val_loss: 0.7528
Epoch 8/50
490/490 ━━━━━━━━━━━━━━━━━━━━ 35s 71ms/step - accuracy: 0.6467 - loss: 0.83

In [10]:
from sklearn.metrics import classification_report, accuracy_score

# Dự đoán trên tập kiểm tra
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")


115/115 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step
Classification Report:
              precision    recall  f1-score   support

         NEG       0.69      0.80      0.74      1318
         NEU       0.40      0.12      0.19       937
         POS       0.68      0.89      0.77      1416

    accuracy                           0.66      3671
   macro avg       0.59      0.60      0.57      3671
weighted avg       0.61      0.66      0.61      3671

Accuracy: 66.11%


In [11]:

# 9. Lưu mô hình
model.save('models.h5')